In [1]:
import os

cwd = os.getcwd()

In [2]:
import pandas as pd


df = pd.read_csv(cwd + '/data/processed/posts.csv')

In [3]:
df.head()

,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text
0,11227809,Why is processing a sorted array faster than p...,<p>Here is a piece of C++ code that shows some...,<java><c++><performance><cpu-architecture><bra...,2012-06-27 13:51:36,2012-06-27 13:56:42,11227902,11227902,26631,why is processing a sorted array faster than p...,process sort array faster process unsort array...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,2009-05-29 18:09:14,2009-05-29 18:13:42,927386,927386,24839,how do i undo the most recent local commits in...,undo recent local commit git accident commit w...
2,2003505,How do I delete a Git branch locally and remot...,<h4>Failed Attempts to Delete a Remote Branch:...,<git><version-control><git-branch><git-push><g...,2010-01-05 01:12:15,2010-01-05 01:13:55,2003515,2003515,19584,how do i delete a git branch locally and remot...,delet git branch local remot fail attempt dele...
3,292357,What is the difference between 'git pull' and ...,"<p>What are the differences between <a href=""h...",<git><version-control><git-pull><git-fetch>,2008-11-15 09:51:09,2008-11-15 09:52:40,292359,292359,13375,what is the difference between git pull and gi...,differ git pull git fetch differ git pull git ...
4,231767,"What does the ""yield"" keyword do?",<p>What is the use of the <code>yield</code> k...,<python><iterator><generator>,2008-10-23 22:21:11,2008-10-23 22:48:44,231855,231855,12274,what does the yield keyword do what is the use...,yield keyword use yield keyword python exampl ...


## Criando os targets

In [4]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [5]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

In [6]:
X, y = df['complete_text'], df['targets'].apply(lambda x: x[0])

In [7]:
y

0                  java
1                   git
2                   git
3                   git
4                python
              ...      
105598     unit-testing
105599                r
105600    asp.net-mvc-4
105601       postgresql
105602       javascript
Name: targets, Length: 105603, dtype: object

In [8]:
all_targets = df.targets.apply(lambda x: x[0]).tolist()

In [9]:
from itertools import chain


targets_set = list()

# for targets in all_targets:
#     for target in targets:
#         targets_set.append(target)
        
for targets in all_targets:
    targets_set.append(targets)

In [10]:
targets_set = set(targets_set)

In [11]:
len(targets_set)

1693

In [12]:
targets_set

{'gitkraken',
 'bootstrapping',
 'excel-2010',
 'slice',
 'internet-explorer-7',
 'netcoreapp3.1',
 'maven',
 'ssl-certificate',
 'css',
 'tfsbuild',
 'math',
 'export',
 'windows-server-2008',
 'separation-of-concerns',
 'tomcat6',
 'tree',
 'salt-stack',
 'collections',
 'user-agent',
 'openafs',
 'google-admin-sdk',
 'plone',
 'graphviz',
 'sql-server-profiler',
 'android-activity',
 'rx-swift',
 'cmake',
 'google-cloud-storage',
 'scp',
 'odata',
 'special-characters',
 'compiler-errors',
 'payment-gateway',
 'nlp',
 'visual-studio',
 'presto',
 'cqrs',
 'https',
 'yaml',
 '3d',
 'metaprogramming',
 'text-files',
 'if-statement',
 'mpi',
 'runtime',
 'grand-central-dispatch',
 'locking',
 'dom-events',
 'aurelia',
 'pattern-matching',
 'laravel-5.4',
 'ruby-on-rails-3',
 'http',
 'hdf5',
 'callback',
 'mp3',
 'google-cloud-firestore',
 'keystore',
 'legacy',
 'switch-statement',
 'threadpool',
 'gitk',
 'memory-leaks',
 'line',
 'text-processing',
 'google-compute-engine',
 'mousew

In [13]:
words_post = df.normalized_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
vector = vectorizer.transform(words_post)

In [14]:
import gc

gc.collect()

0

In [15]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(max_features="auto", n_jobs=1,random_state=44)
rf_model = RandomForestClassifier(max_features="auto", n_estimators=100,max_depth=48, n_jobs=1,random_state=44)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=48, n_jobs=1, random_state=44)

In [17]:
y_pred = rf_model.predict(X_test)

In [18]:
gc.collect()

24

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/home/michelly/anaconda3/envs/deep-learning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                             precision    recall  f1-score   support

                  .htaccess       0.00      0.00      0.00         3
                       .net       1.00      0.01      0.02       166
                   .net-3.5       0.00      0.00      0.00         1
                   .net-4.0       0.00      0.00      0.00         1
                     .net-5       0.00      0.00      0.00         1
              .net-assembly       0.00      0.00      0.00         1
                  .net-core       0.00      0.00      0.00         1
       abstract-syntax-tree       0.00      0.00      0.00         1
             actionscript-3       0.00      0.00      0.00         1
           active-directory       0.00      0.00      0.00         1
                      agile       0.00      0.00      0.00         3
                    airflow       0.00      0.00      0.00         2
                       ajax       0.00      0.00      0.00        12
                  alamofire      

/home/michelly/anaconda3/envs/deep-learning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michelly/anaconda3/envs/deep-learning/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# rf_model.base_estimator_
# rf_model.estimators_
len(rf_model.estimators_)

100

In [21]:
# import pickle
# s = pickle.dumps(rf_model)
# rf_model_2 = pickle.loads(s)

In [22]:
# from joblib import dump, load
# dump(rf_model, 'rf_model.joblib') 

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

np.mean(cross_val_score(rf_model, X_train, y_train, cv=5))